<a href="https://colab.research.google.com/github/TheClassicTechno/eegml_work/blob/main/demo_slideshow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install eeghdf

!git clone https://github.com/eegml/eeghdf.git 
!cd eeghdf

%pip install ipywidgets 
%pip install widgetsnbextension

!pip install git+https://github.com/eegml/eegvis.git

!git clone https://github.com/eegml/eegvis.git
!cd eegvis

!pip install -U git+https://github.com/eegml/eegvis.git
!pip install -U /content/eegvis

import eeghdf
import gzip
import numpy as np
!pip install -U git+https://github.com/eegml/eegvis.git
!pip install mne_features
import matplotlib
import matplotlib.pyplot as plt
import mne_features
import eegvis.stacklineplot as stacklineplot
import eegvis.utils
eegvis.utils.wide_notebook()
import eegml_signal.filters as esfilters

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'eeghdf'...
remote: Enumerating objects: 663, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 663 (delta 55), reused 100 (delta 39), pack-reused 536
Receiving objects: 100% (663/663), 7.92 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (341/341), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eegml/eegvis.git to /tmp/pip-req-build-pcgctl7q
  Running command git clone -q https://github.com/eegml/eegvis.git /tmp/pip-req-build-pcgctl7q
  Installing build dependencies ... don

In [ ]:
!pip install dynaconf

In [ ]:
# to convert to notebook:  pip install jupytext; jupytext --to ipynb demo-slideshow2.py
# to sync: jupytext --sync demo-slideshow2.ipynb
import eegvis, eegvis.utils
# visualization inside the notebook
import eegvis.stacklineplot as eslp
import eegvis.stacklineplot as eslt
import matplotlib.pyplot as plt
import eegvis.montageview

import eeghdf
import numpy as np
from numpy.random import default_rng
nprng = default_rng(seed=42)

import pickle
import copy
from sklearn.utils import resample
import joblib, cloudpickle

from dynaconf import Dynaconf

settings = Dynaconf(
    # envvar_prefix="EEGML",
    settings_files=["settings.toml", ".secrets.toml"],
    environments=True,
    load_dotenv=True,
)



from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    make_scorer,
    precision_score,
    recall_score,
)
import os
import os.path as op

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import (
    StratifiedKFold,
    cross_validate,
    cross_val_score,
    train_test_split,
)
from sklearn.pipeline import Pipeline

from mne_features.feature_extraction import FeatureExtractor, extract_features

from pathlib import Path



def confusion_matrix_scorer(clf, X, y):
    """
    >>> cv_results = cross_validate(svm, X, y, cv=5,
    ...                             scoring=confusion_matrix_scorer)
    >>> # Getting the test set true positive scores
    >>> print(cv_results['test_tp'])
    [10  9  8  7  8]
    >>> # Getting the test set false negative scores
    >>> print(cv_results['test_fn'])
    [0 1 2 3 2]
    """
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    return {"tn": cm[0, 0], "fp": cm[0, 1], "fn": cm[1, 0], "tp": cm[1, 1]}


def multi_scorer(clf, X, y):
    """
    >>> cv_results = cross_validate(svm, X, y, cv=5,
    ...                             scoring=confusion_matrix_scorer)
    >>> # Getting the test set true positive scores
    >>> print(cv_results['test_tp'])
    [10  9  8  7  8]
    >>> # Getting the test set false negative scores
    >>> print(cv_results['test_fn'])
    [0 1 2 3 2]
    """
    y_pred = clf.predict(X)
    accuracy = accuracy_score(y, y_pred)
    cm = confusion_matrix(y, y_pred)
    precision = cm[1, 1] / (cm[1, 1] + cm[0, 1])  # TP/(TP+FP)
    recall = cm[1, 1] / (cm[1, 1] + cm[1, 0])  # TP/(TP+FN)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "tn": cm[0, 0],
        "fp": cm[0, 1],
        "fn": cm[1, 0],
        "tp": cm[1, 1],
    }


def print_multi_scores(model, X, Y):
    score_raw = multi_scorer(model, X, Y)
    N = len(Y)
    for kk, val in score_raw.items():
        if kk not in ["accuracy", "precision", "recall"]:
            print(f"{kk}: {val} ({val/N:.2} fract of total)")
        else:
            print(f"{kk}: {val:.2}")


%load_ext autoreload
%autoreload 2

In [ ]:
eegvis.utils.wide_notebook()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#processing eegs
eeg_path = '/content/drive/MyDrive/hdf'


In [ ]:
#import stevenson_processing as stvp


# EEGML Tools
###  bringing electroencephalogram analysis into the modern era of machine learning

**Khaled Saab, Chris Lee-Messer,
Siyi Tang, Chris Re, and Daniel Rubin**
<p>
<em style="font-size:3.5rem"> QIAI Lab </em> </p>


# EEG images ongoing electrical brain activity
<center><img src="./slideshow/eeg-source-image.png"></center>

<img src="./slideshow/spatio-temporal-eeg1.png">

<img src="./slideshow/spatio-temporal-eeg-trace.png">

# what is needed for typical ML 

### Need tools specialized to the EEG domain, that did not exist.
- EEG Data is scatter across 2 dozen proprietary formats
- Known standards EDF are inconsistently implemented

In [ ]:
USE_ALL = True
if USE_ALL:
    selected_eegs = np.arange(1, 80)  # (1,80) for all of them
    DATA_CACHES = {'x_data': 'x_data-12sPhys-all.pkl', 
    'x_mapping': 'x_mapping-12sPhys-all.pkl',
    'model_unbal': 'model-unbal-all.pkl',
    'model_bal': 'model-bal-all.pkl'}
else:
    selected_eegs = np.arange(1,20+1)
    DATA_CACHES = {'x_data': 'x_data-12sPhys-20eegs.pkl',
                   'x_mapping': 'x_mapping-12sPhys-20eegs.pkl',
                   'model_unbal': 'model-unbal-20eegs.pkl',
                   'model_bal' : 'model-bal-20eegs.pkl'}

In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/hdf_practice/')
print(sys.path)

In [ ]:
# %%
#import /content/drive/MyDrive/hdf_practice/add_annotations_function.py
#import add_annotations_function
import pandas as pd
import numpy as np
import h5py
import shutil
import dateutil 
eeg_file_path='/content/drive/MyDrive/hdf/'
eeg_annot_path='/content/drive/MyDrive/hdf_practice/juliaannotations/'
start_eeg_num, end_eeg_num = 1,79
eeg_columns_num = list(range(start_eeg_num,end_eeg_num+1)) # for all use list(range(1,79+1))
eeg_columns_str = [str(ii) for ii in eeg_columns_num]
dfa = pd.read_csv(eeg_file_path+'annotations_2017_A.csv', usecols=eeg_columns_str)
dfb = pd.read_csv(eeg_file_path+'annotations_2017_B.csv', usecols=eeg_columns_str)
dfc = pd.read_csv(eeg_file_path+'annotations_2017_C.csv', usecols=eeg_columns_str)
clinical = pd.read_csv(eeg_file_path+'clinical_information.csv') # remember ID, subject and eeg num start at 1, so use iloc[eeg_num -1]

In [ ]:
eeg_file_path='/content/drive/MyDrive/hdf/'
eeg_annot_path='/content/drive/MyDrive/hdf_practice/juliaannotations/'
eeg_num =1
shutil.copy(eeg_file_path+f"eeg{eeg_num}.eeg.h5", eeg_annot_path+f"eeg{eeg_num}.annot.eeg.h5") # !cp eeg{eeg_num}.eeg.h5 eeg{eeg_num}.annot.eeg.h5

In [ ]:
eeg_annot_path='/content/drive/MyDrive/hdf_practice/juliaannotations/'
hf = h5py.File(eeg_annot_path+f'eeg{eeg_num}.annot.eeg.h5', 'r+')
hf

In [ ]:
eeghdfs=[]
annot=[]
'''
for eeg_num in range(start_eeg_num,end_eeg_num+1):
  eeghdfs.append(eeg_file_path+f'eeg{eeg_num}.eeg.h5')
  annot.append(eeg_annot_path+f'eeg{eeg_num}.annot.eeg.h5')
'''

for eeg_num in range(start_eeg_num,end_eeg_num+1):
  EEGFILE=os.path.join(eeg_file_path, f"eeg{eeg_num}.eeg.h5")
  ANNOT_EEGFILE=os.path.join(eeg_annot_path, f"eeg{eeg_num}.annot.eeg.h5")
  
  eeg = eeghdf.Eeghdf(EEGFILE)
  samp_freq=eeg.sample_frequency

  eeg_ANNOT = eeghdf.Eeghdf(ANNOT_EEGFILE)
  samp_freq_ANNOT=eeg_ANNOT.sample_frequency
  eeghdfs.append(eeg)
  annot.append(eeg)

file_name = eeghdfs[0].file_name

In [ ]:
print(eeghdfs)

In [ ]:
'''
for eeg_num in range(start_eeg_num,end_eeg_num+1):
  eeghdfs, annot = add_annotations_function.add_annotations(eeg_num)
annot
file_name = eeghdfs[0].file_name
'''

First need something like DICOM designed for ML

In [ ]:
eeghdfs[0]

In [ ]:
import eeghdf 
hf = eeghdfs[0] # loaded 
s_freq = hf.sample_frequency
f"{Path(hf.file_name).name}, {hf.sample_frequency}"

In [ ]:
hf.phys_signals.shape

In [ ]:
# import eeg library for reading EEGs
import eeghdf

# load eeg file
hf = eeghdf.Eeghdf(file_name)
print(f"EEG file: {hf.file_name}\n")

# view EEG values
print(f"EEG values: {hf.phys_signals[5, 1000:1010]}\n")
print(f"Out of core access of 1.78-million-th sample: {hf.phys_signals[5,1780000:1780005]}\n")

# view patient information
print(f"Patient info: {hf.patient}")

In [ ]:
vec = hf.phys_signals[5,1000:20000]
np.sqrt( np.sum(vec*vec)/len(vec))

In [ ]:
# labels are available each second for this dataset
#list(hf.hdf['record-0/extensions/dense_seizure_annotations'].items()) #not for hdf .annot.eeg.h5

### load and chop up EEGs for training

In [ ]:
s_freq = int(hf.sample_frequency)
selected_hdfs = [ef.hdf for ef in eeghdfs]
seg_time = 12
n_channels = 19
sz_agree_criteria = 3
whiten = False
USE_CACHED_DATA = (
    True and Path(DATA_CACHES["x_data"]).is_file()
)  # change this False->True if you want to use the cache

if not USE_CACHED_DATA:
    '''
    x_data, mapping = stvp.process_stevenson_eeghdf_data(
        eeghdfs,
        s_freq=s_freq,
        seg_time=seg_time,
        n_channels=n_channels,
        whiten=False,  # don't whiten the EEG, scale the features
    )
    '''
    x_data, mapping = eeg(
        eeghdfs,
        s_freq=s_freq,
        seg_time=seg_time,
        n_channels=n_channels,
        whiten=False,  # don't whiten the EEG, scale the features
    )
    # cache the result
    with open(DATA_CACHES["x_data"], "wb+") as fp:
        pickle.dump(x_data, fp)
    with open(DATA_CACHES["x_mapping"], "wb+") as fp:
        pickle.dump(mapping, fp)
else:  # use the cached data
    with open(DATA_CACHES["x_data"], "rb") as fp:
        x_data = pickle.load(fp)
    with open(DATA_CACHES["x_mapping"], "rb") as fp:
        mapping = pickle.load(fp)
# mapping array is array of string arrays [filename, start_sec,end_sec]

In [ ]:

ylabels = stvp.process_stevenson_labels(
    annot, selected_eegs, sz_agree_criteria, seg_time
)
# data, ylabels, mapping = stvp.stevenson_eeg_preprocess(settings.EEGML_STEV_NEONATES) # load, whiten/zero-mean waveforms

In [ ]:
data1 = x_data[0,:,:]

In [ ]:
len(data1[5,:])

In [ ]:
x_data.shape, ylabels.shape

In [ ]:
#['app_entropy', 'decorr_time', 'energy_freq_bands', 'higuchi_fd', 'hjorth_complexity', 'hjorth_complexity_spect',
# 'hjorth_mobility', 'hjorth_mobility_spect', 'hurst_exp', 'katz_fd', 'kurtosis', 'line_length', 'mean', 
# 'pow_freq_bands', 'ptp_amp', 'quantile', 'rms', 'samp_entropy', 'skewness', 'spect_edge_freq', 'spect_entropy',
# 'spect_slope', 'std', 'svd_entropy', 'svd_fisher_info', 'teager_kaiser_energy', 'variance', 
# 'wavelet_coef_energy', 'zero_crossings', 'max_cross_corr', 'nonlin_interdep', 'phase_lock_val', 'spect_corr', 
# 'time_corr']
# selected_funcs = ["mean", "std", "ptp_amp", 'line_length' ] # features to use
selected_funcs = [
#   "decorr_time",
#   "energy_freq_bands",
#   "hjorth_complexity",
#   "hjorth_complexity_spect",
#   "hjorth_mobility",
#   "hjorth_mobility_spect",
#   "katz_fd",
    "kurtosis",
#    "line_length",
    "pow_freq_bands",
    "ptp_amp",
    "skewness",
#    "spect_edge_freq",
#    "spect_entropy",
#    "spect_slope",
#    "std",
 #   "teager_kaiser_energy",
 #   "variance",
 #   "wavelet_coef_energy",
 #   "zero_crossings",
]
# data_features = extract_features(x_data, sfreq=s_freq, selected_funcs=selected_funcs) # if wanted to extract features separately from pipeline

In [ ]:
# split train/test
X_train, X_test, y_train, y_test = train_test_split(x_data, ylabels, test_size=0.2)

### Build a model using loaded eeg data

In [ ]:
# if need to used cached model instead of training
if Path(DATA_CACHES['model_unbal']).is_file():
    model_pipe = joblib.load(DATA_CACHES['model_unbal'])

In [ ]:
model_pipe = Pipeline(
    [   # excluding data augmentation step here
        ("fe", FeatureExtractor(sfreq=s_freq, selected_funcs=selected_funcs)),
        ("scaler", StandardScaler()),
        ("clf", RandomForestClassifier(n_estimators=100, max_depth=4)),
    ]
)

model_pipe.fit(X_train, y_train)

In [ ]:
print("training scores")
print_multi_scores(model_pipe, X_train, y_train)

In [ ]:
print("test-scores")
print_multi_scores(model_pipe, X_test, y_test)

In [ ]:
# if need to used cached model instead of training
if Path(DATA_CACHES['model_bal']).is_file():
    model_bal = joblib.load(DATA_CACHES['model_bal'])

### Build a model using processed eeg data (naively balanced, omitting augmentation)

In [ ]:
model_bal = Pipeline(
    [
        ("fe", FeatureExtractor(sfreq=s_freq, selected_funcs=selected_funcs)),
        ("scaler", StandardScaler()),
        ("clf", RandomForestClassifier(n_estimators=100, max_depth=4)),
    ]
)

X_oversampled, y_oversampled = resample(
    X_train[y_train == 0],
    y_train[y_train == 0],
    replace=True,
    n_samples=X_train[y_train == 1].shape[0],
    random_state=123321,
)
#
# Append the oversampled minority class to training data and related labels
#
X_balanced = np.vstack((X_train[y_train == 1], X_oversampled))
y_balanced = np.hstack((y_train[y_train == 1], y_oversampled))

model_bal.fit(X_balanced, y_balanced)

In [ ]:
# check that the dataset really is balanced now
y_balanced.mean(), len(X_balanced), len(y_balanced)

In [ ]:
if Path(DATA_CACHES['model_bal']).is_file(): model_bal = joblib.load(DATA_CACHES['model_bal'])

### model performance for balanced dataset

In [ ]:
print("training scores (balanced data)")
print_multi_scores(model_bal, X_balanced, y_balanced)

In [ ]:
print("original test-set scores for model trained with balanced data")
print_multi_scores(model_bal, X_test, y_test)

In [ ]:
y_pred_all = model_bal.predict(x_data)

In [ ]:
clips_correct = x_data[y_pred_all == ylabels]
clips_errors = x_data[y_pred_all != ylabels]

In [ ]:
clips_errors_y_true = ylabels[y_pred_all != ylabels]
clips_errors_y_pred = y_pred_all[y_pred_all != ylabels]

In [ ]:
# next I want to show a smapling of the corrects an incorrects
clips_correct.shape

In [ ]:
clips_correct_y_true = ylabels[y_pred_all == ylabels]
clips_correct_y_pred = y_pred_all[y_pred_all == ylabels]

In [ ]:
X_train.shape

In [ ]:
# correct = np.arange(20.0) # for testing
# Get a random subsample of correct and incorrect EEG clips
correct_picks = nprng.choice(clips_correct.shape[0], size=9, replace=False)
error_picks = nprng.choice(clips_errors.shape[0], size=9, replace=False)
correct_samples = clips_correct[correct_picks]
correct_samples_ypred = clips_correct_y_pred[correct_picks]
correct_samples_ytrue = clips_correct_y_true[correct_picks]
error_samples = clips_errors[error_picks]
error_samples_ypred = clips_errors_y_pred[error_picks]
error_samples_ytrue = clips_errors_y_true[error_picks]

In [ ]:
list(zip(error_samples_ytrue, error_samples_ypred))

In [ ]:
list(zip(correct_samples_ytrue, correct_samples_ypred))

In [ ]:
# check the variance of an error sample
error_samples
arr = error_samples[0]
np.sqrt(np.mean(arr[5,:]*arr[5,:]))

In [ ]:
arr[5,:]

## Need to be able to do in depth error analysis
### Why is the precision bad?
#### Look at the errors and correctly predicted data using EEG ML specific visualization

In [ ]:
# try single plot
montage_name = 'double banana'
elabels = hf.shortcut_elabels[:19]
if montage_name in eegvis.montageview.MONTAGE_BUILTINS:
    montage_derivation = eegvis.montageview.MONTAGE_BUILTINS[montage_name](elabels)

fig = plt.figure(figsize=(8,6))
ax = fig.subplots(1,1)
ii=0
eeg_arr = correct_samples[ii]
eslp.show_arr_as_montaged_eeg(eeg_arr, montage_derivation,fs=s_freq,ylabels=elabels,ax=ax,ysensitivity=15)
    #faxs[ii].set_title(f"y_true={error_samples_ytrue[ii]}, y_pred={error_samples_ypred[ii]}")
ax.set_title(f"y_true={correct_samples_ytrue[ii]}, y_pred={correct_samples_ypred[ii]}") 
eslp.add_relative_vertical_scalebar(ax, units='$\mu$V')

In [ ]:
elabels = hf.shortcut_elabels[:19]
fig, axs = eslp.show_grid_arr_as_montaged_eeg(error_samples, s_freq, elabels, ytrues=error_samples_ytrue,ypreds=error_samples_ypred, ysensitivity=15.0)

### Correct predictions

In [ ]:
elabels = hf.shortcut_elabels[:19]
fig, axs = eslp.show_grid_arr_as_montaged_eeg(correct_samples, s_freq, elabels, ytrues=correct_samples_ytrue,ypreds=correct_samples_ypred, ysensitivity=15.0)

### if you need to focus in in more detail on a specific model and specific EEG

In [ ]:
# uncomment below to show the demo 

%run eegvis-test-range-viewer.py
gspec.show()

### EEG ML tools built on top of:
- hdf5 file format for storing eeg data: 
  - includes data integrity, compression, out-of-memory access, multiple readers, single-writer
  - easy access on all major 
  - S3 read-only support
- matplotlib, bokeh, panel : 
    - supports working in jupyter,vscode
    - website
    - wasm support added recently gives ability to run most/all code in browser

### Thanks
#### Visit us at the demo table!


### Acknowledgements

These tools were inspired by many works in the lab

We are excited by many directions to improve these tools
- clinician annotation with AI assistance
- active learning
- confidence scores

## Model Caching code

In [ ]:

if False: # change to True when you want to cache a model 
    # cloudpickle/dill not required given using packaged models
    with open(DATA_CACHES['model_unbal']+'.cloudpickle', 'wb+') as fp: cloudpickle.dump(model_pipe, fp)
    with open(DATA_CACHES['model_bal']+'.cloudpickle', 'wb+') as fp: cloudpickle.dump(model_bal, fp)
    joblib.dump(model_pipe, DATA_CACHES['model_unbal'])
    joblib.dump(model_bal, DATA_CACHES['model_bal'])

## Model load from cache


In [ ]:
if False:
    if Path(DATA_CACHES['model_unbal']).is_file():
        model_pipe = joblib.load(DATA_CACHES['model_unbal'])

    if Path(DATA_CACHES['model_bal']).is_file():
        model_bal = joblib.load(DATA_CACHES['model_bal'])